У таблиц ratings и movies есть общий столбец movieId, который каждому фильму из таблицы movies ставит в соответствие поставленные ему оценки из таблицы ratings. Мы хотим объединить их в единую структуру согласно этому соответствию. Объединения такого рода часто называют объединением по ключевому столбцу.

<div style="background-color: #f5f5f5; padding: 15px; color: black; width: 80%;">→ Однако прежде чем мы перейдём к дальнейшей работе с нашими таблицами о фильмах, мы должны рассмотреть основные типы объединения таблиц.</div>

## Типы объединений

Типы объединений в Pandas тесно связаны с операцией join из SQL, которую мы будем рассматривать в курсе в дальнейшем.

Они представлены на схеме ниже в виде кругов Эйлера. (Круги Эйлера — это геометрический способ отобразить отношения между множествами.)

<img src='../static/img/pandas_adv_8.png'>

Прежде чем мы перейдём к дальнейшей работе с таблицами о фильмах, рассмотрим два основных типа объединения таблиц:

### inner (внутреннее)

При использовании такого типа объединения в результирующей таблице остаются только те записи, которые есть в обеих таблицах.<br>
<em>Аналогия в теории множеств: Пересечение (intersection) множеств А и В.</em>

Строки, для которых совпадение не было найдено, удаляются.

### outer (внешнее)

Данный тип делится на три подтипа:

- full — используется как outer по умолчанию, объединяет все варианты в обеих таблицах.<br>
<i>Аналогия в теории множеств: Объединение (union) множеств А и В.</i>

- left — для всех записей из «левой» таблицы (например, ratings) ведётся поиск соответствий в «правой» (например, movies). В результирующей таблице останутся только те значения, которым были найдены соответствия, то есть только значения из ratings.<br>
<i>Аналогия в теории множеств: Вычитание (difference) множества B из результата объединения (union) множеств А и В.</i>

- right — аналогично предыдущему, но остаются значения только из «правой» таблицы. <br>
<i>Аналогия в теории множеств: Вычитание (difference) множества А из результата объединения (union) множеств А и В.</i>

Во всех трёх случаях, если совпадений между таблицами не найдено, на этом месте ставится пропуск (NaN).

## Метод объединения join

Для объединения двух таблиц по индексам используется метод DataFrame <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html">join()</a>. Однако данный метод можно применить и для того, чтобы объединить таблицы по ключевому столбцу (в нашем случае это movieId).

#### Основные параметры метода join()

- other — таблица, которую мы присоединяем. При объединении она является «правой», а исходная таблица, от имени которой вызывается метод, является «левой».
- how — параметр типа объединения. Он может принимать значения 'inner', 'left' (left outer), 'right' (right outer), и 'outer' (full outer). По умолчанию параметр установлен на 'left'.
- on — параметр, который определяет, по какому столбцу в «левой» таблице происходит объединение по индексам из «правой».
- lsuffix и rsuffix — дополнения (суффиксы) к названиям одноимённых столбцов в «левой» и «правой» таблицах.

Ниже представлена общая схема работы метода join() в зависимости от типа объединения:

<img src="../static/img/pandas_adv_9.png" style="width: 80%; background-color: white; padding: 10px;">

<div style="background-color: #e0ffd1;color: black;border: 3px solid black; padding: 15px; margin-right: 500px; width: 80%;">Если использовать метод join() «в лоб» (без указания ключевого столбца), то объединение произойдёт, как и задумано — по индексам двух таблиц согласно установленному типу объединения.</div>

Проверим это, объединив таблицы типом left. Так как в наших таблицах есть одноимённые столбцы, установим один из суффиксов, чтобы избежать ошибки:

In [1]:
# code from other notebook
import pandas as pd

movies_df = pd.read_csv("./data/movies.csv")
ratings_df1 = pd.read_csv("./data/ratings1.csv")
dates_df = pd.read_csv("./data/dates.csv")
dates_df["date"] = pd.to_datetime(dates_df["date"])
ratings_df2 = pd.read_csv("./data/ratings2.csv")
ratings = pd.concat([ratings_df1, ratings_df2], ignore_index=True)
ratings = ratings.drop_duplicates(ignore_index=True)
ratings_dates = pd.concat([ratings, dates_df], axis=1)

In [2]:
joined_false = ratings_dates.join(movies_df, rsuffix="_right", how="left")
joined_false.head()

,userId,movieId,rating,date,movieId_right,title,genres
0,1,1,4.0,2000-07-30 18:45:03,1.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,2.0,Jumanji (1995),Adventure|Children|Fantasy
2,1,6,4.0,2000-07-30 18:37:04,3.0,Grumpier Old Men (1995),Comedy|Romance
3,1,47,5.0,2000-07-30 19:03:35,4.0,Waiting to Exhale (1995),Comedy|Drama|Romance
4,1,50,5.0,2000-07-30 18:48:51,5.0,Father of the Bride Part II (1995),Comedy


При объединении таблиц по индексам в результирующую таблицу попали все строки из «левой» таблицы, а недостающие строки из «правой» были заполнены пропусками. Так работает тип объединения left.

Попробуйте изменить тип объединения, чтобы посмотреть на разницу результирующих таблиц.

In [3]:
joined_test = ratings_dates.join(movies_df, rsuffix="_right", how="inner")
joined_test.head()

,userId,movieId,rating,date,movieId_right,title,genres
0,1,1,4.0,2000-07-30 18:45:03,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,2,Jumanji (1995),Adventure|Children|Fantasy
2,1,6,4.0,2000-07-30 18:37:04,3,Grumpier Old Men (1995),Comedy|Romance
3,1,47,5.0,2000-07-30 19:03:35,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,1,50,5.0,2000-07-30 18:48:51,5,Father of the Bride Part II (1995),Comedy


<div style="background-color: #e0ffd1;color: black;border: 3px solid black; padding: 15px; margin-right: 500px; width: 80%;">Обратите внимание, что в данном случае у нас получилось два столбца, соответствующих идентификатору фильма: один — из «левой» таблицы (movieId), а другой — из «правой» (movieId_right).</div>

Однако это не тот результат, который мы хотели, ведь мы не получили соответствия фильмов и их рейтингов. Чтобы совместить таблицы по ключевому столбцу с помощью метода join(), необходимо использовать ключевой столбец в «правой» таблице в качестве индекса. Это можно сделать с помощью метода set_index(). Также необходимо указать название ключа в параметре on.

In [4]:
joined = ratings_dates.join(movies_df.set_index("movieId"), on="movieId", how="left")
joined.head()

,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


В результате такого объединения для каждого идентификатора фильма movieId в таблице ratings_dates найден совпадающий с ним идентификатор movieId в таблице movies и присоединена информация о самом фильме (title и genres). Это как раз то, что нам нужно.

<div style="background-color: #e0ffd1;color: black;border: 3px solid black; padding: 15px; margin-right: 500px; width: 80%;">Обратите внимание, что в результате такого объединения остался лишь один столбец movieId.</div>

<div style="background-color: #e0ffd1;color: black;border: 3px solid black; padding: 15px; margin-right: 500px; width: 80%;">Примечание. Join() также можно использовать с параметром how='outer'.</div>

## Метод объединения MERGE

Аналогично предыдущему, метод <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html">merge()</a> предназначен для слияния двух таблиц по ключевым столбцам или по индексам. Однако, в отличие от join(), метод merge() предлагает более гибкий способ управления объединением, благодаря чему является более популярным.

### Основные параметры метода merge()

- right — присоединяемая таблица. По умолчанию она является «правой».
- how — параметр типа объединения. По умолчанию принимает значение 'inner'.
- on — параметр, который определяет, по какому столбцу происходит объединение. Определяется автоматически, но рекомендуется указывать вручную.
- left_on — если названия столбцов в «левой» и «правой» таблицах не совпадают, то данный параметр отвечает за наименования ключевого столбца исходной таблицы.
- right_on — аналогично предыдущему, параметр отвечает за наименование ключевого столбца присоединяемой таблицы.

<div style="background-color: #f5f5f5; padding: 15px; color: black; width: 80%;">→ Метод merge() в первую очередь предназначен для слияния таблиц по заданным ключам, поэтому он не требует установки ключевых столбцов в качестве индекса присоединяемой таблицы. Кроме того, данный метод позволяет объединять даже таблицы с разноимёнными ключами. Таким образом, merge() проще в использовании и более многофункционален, чем схожие методы.</div>

Посмотрим на метод merge() в действии. Произведём слияние наших таблиц и получим ту же таблицу, что и ранее:

In [13]:
ratings_dates.head()

,userId,movieId,rating,date
0,1,1,4.0,2000-07-30 18:45:03
1,1,3,4.0,2000-07-30 18:20:47
2,1,6,4.0,2000-07-30 18:37:04
3,1,47,5.0,2000-07-30 19:03:35
4,1,50,5.0,2000-07-30 18:48:51


In [14]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [15]:
merged = ratings_dates.merge(movies_df, on="movieId", how="left")
merged.head()

,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


Проверим, что число строк в таблице ratings_dates совпадает с числом строк в результирующей таблице merged:

In [6]:
print("Число строк в таблице ratings_dates:", ratings_dates.shape[0])
print("Число строк в таблице merged:", merged.shape[0])

Число строк в таблице ratings_dates: 100836
Число строк в таблице merged: 100836


Всё прошло успешно: для каждой оценки пользователя мы нашли информацию о фильме, которому она была выставлена.

## Особенности использования merge()

Возникает вопрос: почему мы выбрали тип объединения left, а не full, например?

Найти ответ нам поможет пример. Объединим ratings_dates с movies по ключевому столбцу movieId, но с параметром how='outer' (full outer) и выведем размер таблицы, а также её «хвост»:

In [7]:
merged2 = ratings_dates.merge(movies_df, how="outer", on="movieId")
merged2.tail()

,userId,movieId,rating,date,title,genres
100849,NaN,30892,NaN,NaT,In the Realms of the Unreal (2004),Animation|Documentary
100850,NaN,32160,NaN,NaT,Twentieth Century (1934),Comedy
100851,NaN,32371,NaN,NaT,Call Northside 777 (1948),Crime|Drama|Film-Noir
100852,NaN,34482,NaN,NaT,"Browning Version, The (1951)",Drama
100853,NaN,85565,NaN,NaT,Chalet Girl (2011),Comedy|Romance


Результирующее число строк в таблице увеличилось. Но за счёт чего?

Оказывается, в таблице movies содержались фильмы, которым ещё не были выставлены оценки. В результате объединения типом full outer информация о фильмах перенеслась из таблицы movies в результирующую таблицу. Однако, поскольку оценки фильмам ещё не были выставлены, соответствующие столбцы таблицы ratings_dates заполнились пропусками (NaN). Такие фильмы были записаны в конец таблицы.

<div style="background-color: #e0ffd1;color: black;border: 3px solid black; padding: 15px; margin-right: 500px; width: 80%;">Важно! Учитывайте такие нюансы при работе с несколькими таблицами и всегда проверяйте результат объединения.</div>

---
<div style="background-color: #f5f5f5; padding: 15px; color: black; width: 80%;">
→ Метод merge() с внешним (outer) типом объединения может использоваться как аналог метода concat() при объединении таблиц с одинаковой структурой (одинаковые количество и названия столбцов) по строкам. В таком случае все одноимённые столбцы таблиц будут считаться ключевыми.
</div>

Рассмотрим пример: объединим таблицы ratings1 и ratings2, как мы уже делали раньше, но теперь используем метод merge():

In [8]:
merge_ratings = ratings_df1.merge(ratings_df2, how="outer")
print("Число строк в итоговой таблице:", merge_ratings.shape[0])
merge_ratings.tail()

Число строк в итоговой таблице: 100836


,userId,movieId,rating
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0
100835,610,170875,3.0


<div style="background-color: #e0ffd1;color: black;border: 3px solid black; padding: 15px; margin-right: 500px; width: 80%;">Обратите внимание, что при использовании метода merge() для склейки двух таблиц у нас автоматически пропали дубликаты, которые мы видели при использовании метода concat(). Это особенность метода merge() — автоматическое удаление дублей.</div>

## Какой метод объединения использовать?

Итак, мы рассмотрели три основных метода объединения таблиц: concat(), join() и merge(). Давайте структурируем материал, изложенный ранее, в виде небольшой блок-схемы, которая поможет вам определить, какой метод является предпочтительным при объединении таблиц.

<img src='../static/img/pandas_adv_10.png' style='width: 80%;'>

<div style="border: 1px solid white; padding: 5px; margin-right: auto;  width: 80%;"> ✍ А теперь предлагаем вам самим потренироваться в использовании методов join() и merge() ↓</div>

###  Задание 7.1

Какой параметр методов join() и merge() отвечает за используемый тип объединения?
- type_union
- how
- on
- prefix 

<details>
<summary><strong>Show answer</strong> (Click Here)</summary>
    &emsp; &emsp; <code>
how
</code>
</details>

###   Задание 7.2

В <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html">документации по методу merge()</a> найдите параметры, которые позволяют управлять объединением таблиц по индексам:
- left_idx, right_idx
- l_suffix, r_suffix
- left_index, right_index
- left_on, right_on 
<details>
<summary><strong>Show answer</strong> (Click Here)</summary>
    &emsp; &emsp; <code>
Верно: Для объединения двух таблиц по индексам с помощью метода merge() необходимо установить значение параметров left_index и right_index на True.
</code>
</details>

###    Задание 7.3


Дано две исходных таблицы:<br>
<code>
data_1 = pd.DataFrame({'Value': [100, 45, 80],
                       'Group': [1, 4, 5]},
                      index = ['I0', 'I1', 'I2']
                     )
</code><br>
<img src="../static/img/pandas_adv_11.png">
<br>
<code>
data_2 = pd.DataFrame({'Company': ['Google', 'Amazon', 'Facebook'],
                       'Add': ['S0', 'S1', 'S7']},
                      index = ['I0', 'I1', 'I3']
                     )
</code><br>
<img src="../static/img/pandas_adv_12.png">
<br>
Какой из перечисленных вариантов кода позволяет получить представленный ниже результат?<br>

<img src="../static/img/pandas_adv_13.png">
<br>


- data_1.join(data_2, how='inner')

- data_1.join(data_2, how='outer')

- data_1.join(data_2, how='left')

- data_1.join(data_2, how='right')



<details>
<summary><strong>Show answer</strong> (Click Here)</summary>
    &emsp; &emsp; <code>
data_1.join(data_2, how='inner')
</code>
</details>

###    Задание 7.4

Даны две исходные таблицы:

- a = pd.DataFrame({'A': ['a', 'b', 'c'], 'B': [103, 214, 124], 'C': [1, 4, 2]})

- b = pd.DataFrame({'V': ['d', 'b', 'c'], 'U': [1393.7, 9382.2, 1904.5], 'C': [1, 3, 2]})

Какой из перечисленных вариантов кода позволяет получить представленный ниже результат?
<br>
<img src="../static/img/pandas_adv_14.png" style='width: 400px;'>
<br>


- a.join(b, how='inner', r_suffix='_r')

- a.merge(b, how='left', on='C')

- a.merge(b, how='inner', right_on='А', left_on='V')

- b.join(a.set_index('C'), how='right', on='C')

- a.merge(b, how='right', on='C')

- a.merge(b, how='inner', on='C')


<details>
<summary><strong>Show answer</strong> (Click Here)</summary>
    &emsp; &emsp; <code>
- a.merge(b, how='right', on='C')
</code>
</details>

###    Задание 7.4

Даны две таблицы: items_df, в которой содержится информация о наличии товаров на складе, и purchase_df с данными о покупках товаров.

Первая:

    items_df = pd.DataFrame({
        'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 100132, 312394],
        'vendor': ['Samsung', 'LG', 'Apple', 'Apple', 'LG', 'Apple', 'Samsung', 'Samsung', 'LG', 'ZTE'],
        'stock_count': [54, 33, 122, 18, 102, 43, 77, 143, 60, 19]
    })

Вторая:

    purchase_df = pd.DataFrame({
        'purchase_id': [101, 101, 101, 112, 121, 145, 145, 145, 145, 221],
        'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 103845, 100132],
        'price': [13900, 5330, 38200, 49990, 9890, 33000, 67500, 34500, 89900, 11400]
    })

Информация в таблицах представлена в виде следующих столбцов:

- item_id — идентификатор модели;
- vendor — производитель модели;
- stock_count — имеющееся на складе количество данных моделей (в штуках);
- purchase_id — идентификатор покупки;
- price — стоимость модели в покупке.

Вам необходимо сделать следующее:

Сформируйте DataFrame merged, так чтобы после объединения purchase_df и items_df остались модели, которые учтены на складе и имели продажи.

На основе таблицы merged найдите суммарную выручку, которую можно было бы получить от продажи всех товаров, имеющихся на складе. Результат занесите в переменную income.


<details>
<summary><strong>Show answer</strong> (Click Here)</summary>
    &emsp; &emsp; <code>
- a.merge(b, how='right', on='C')
</code>
</details>

In [9]:
items_df = pd.DataFrame(
    {
        "item_id": [
            417283,
            849734,
            132223,
            573943,
            19475,
            3294095,
            382043,
            302948,
            100132,
            312394,
        ],
        "vendor": [
            "Samsung",
            "LG",
            "Apple",
            "Apple",
            "LG",
            "Apple",
            "Samsung",
            "Samsung",
            "LG",
            "ZTE",
        ],
        "stock_count": [54, 33, 122, 18, 102, 43, 77, 143, 60, 19],
    }
)
items_df

,item_id,vendor,stock_count
0,417283,Samsung,54
1,849734,LG,33
2,132223,Apple,122
3,573943,Apple,18
4,19475,LG,102
5,3294095,Apple,43
6,382043,Samsung,77
7,302948,Samsung,143
8,100132,LG,60
9,312394,ZTE,19


In [10]:
purchase_df = pd.DataFrame(
    {
        "purchase_id": [101, 101, 101, 112, 121, 145, 145, 145, 145, 221],
        "item_id": [
            417283,
            849734,
            132223,
            573943,
            19475,
            3294095,
            382043,
            302948,
            103845,
            100132,
        ],
        "price": [13900, 5330, 38200, 49990, 9890, 33000, 67500, 34500, 89900, 11400],
    }
)
purchase_df

,purchase_id,item_id,price
0,101,417283,13900
1,101,849734,5330
2,101,132223,38200
3,112,573943,49990
4,121,19475,9890
5,145,3294095,33000
6,145,382043,67500
7,145,302948,34500
8,145,103845,89900
9,221,100132,11400


In [11]:
merged = items_df.merge(purchase_df, on="item_id", how="inner")
merged

,item_id,vendor,stock_count,purchase_id,price
0,417283,Samsung,54,101,13900
1,849734,LG,33,101,5330
2,132223,Apple,122,101,38200
3,573943,Apple,18,112,49990
4,19475,LG,102,121,9890
5,3294095,Apple,43,145,33000
6,382043,Samsung,77,145,67500
7,302948,Samsung,143,145,34500
8,100132,LG,60,221,11400


In [12]:
income = (merged["stock_count"] * merged["price"]).sum()
income

19729490